In [7]:
import warnings
warnings.filterwarnings(action='ignore')
%config Completer.use_jedi = False

import pandas as pd

In [8]:
df1 = pd.read_csv('./data/NSeries_최신순_5,000.txt', index_col=0)
df2 = pd.read_csv('./data/NSeries_인기순_5,000.txt', index_col=0)
##df3 = pd.read_csv('./data/NSeries_Month100.txt', index_col=0)
#df4 = pd.read_csv('./data/NSeries_Now100.txt', index_col=0)

In [9]:
df = pd.concat([df1, df2], axis=0, ignore_index=1)
df.index = df.index+1
df

,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,불로불사로 산다는 것은,판타지,겨울에어컨,(2023.11.29.),(2024.06.27.),196,"1,710",23,8.8
2,천마홈즈 런던앙복 [독점],무협,옴니버,(2023.03.28.),(2024.06.27.),394,"5,977",125,8.7
3,이름대로 살련다,현판,SSANTA,(2023.12.06.),(2024.06.27.),151,40,2,9.3
4,부패의 사제,판타지,사다듬,(2020.12.17.),(2024.06.27.),375,"3,587",116,8.8
5,불로불사가 사는 법 [독점],현판,Neuf,(2023.01.24.),(2024.06.27.),409,"4,169",30,9.3
...,...,...,...,...,...,...,...,...,...
9996,바람의 마스터 [단행본],현판,임영기,(2016.08.12.),(2016.08.12.),9,20,8,6.4
9997,녹림마존,무협,정재욱,(2015.12.21.),(2016.05.25.),127,4,6,5.3
9998,19금,19금,19금,19금,19금,19금,19금,19금,19금
9999,만겁강호 [단행본],무협,구당협,(2015.03.19.),(2015.03.19.),8,7,8,5.8


In [10]:
# 제목에서 선독점 같은 상관없는 자료 제거
df.제목 = df.제목.str.replace('[단행본]', '').str.strip()
df.제목 = df.제목.str.replace('[독점]', '').str.strip()
df.제목 = df.제목.str.replace('선공개', '').str.strip()
df.제목 = df.제목.str.replace(r'\[|\]', '').str.strip()

In [11]:
# 제목이 중복되는 자료 제거
df=df.drop_duplicates('제목', keep='first', ignore_index=True)
df.제목.value_counts() # 중복값 확인

불로불사로 산다는 것은              1
맹주                        1
만렙 플레이어                   1
생사신의                      1
강철 상남자                    1
                         ..
이걸 실장님이? : 아이돌 방생 프로젝트    1
무한 인벤토리 들고 무한회귀           1
세상을 구해 볼 생각이다             1
내 머릿속 소드마스터               1
고검잔향                      1
Name: 제목, Length: 6790, dtype: int64

In [12]:
# 19금 제거
index=df.loc[(df.제목 == '19금') | (df.제목 == '판매중지')].index
index
df.drop(index, inplace=True)

#df.회수 = df.회수.str.replace(',', '')
df.회수 = df.회수.astype(int)
df.댓글수 = df.댓글수.str.replace(',', '')
df.댓글수 = df.댓글수.astype(int)
df.추천수 = df.추천수.str.replace(',', '')
df.loc[df.추천수 == '좋아요', '추천수'] = 0
df.추천수 = df.추천수.astype(int)
df.추천수 = df.추천수.astype(int)
#df.평점 = df.평점.str.replace(',', '')
df.평점 = df.평점.astype(float)


df['연재 시작일'] = df['연재 시작일'].str.replace(r'[()]', '').str.strip()
df['마지막 연재일'] = df['마지막 연재일'].str.replace(r'[()]', '').str.strip()

df['연재 시작일'] = pd.to_datetime(df['연재 시작일'])
df['마지막 연재일'] = pd.to_datetime(df['마지막 연재일'])

In [13]:
df = df.reset_index(drop=True)
df.index = df.index+1
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6788 entries, 1 to 6788
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   제목       6788 non-null   object        
 1   장르       6788 non-null   object        
 2   작가       6788 non-null   object        
 3   연재 시작일   6788 non-null   datetime64[ns]
 4   마지막 연재일  6788 non-null   datetime64[ns]
 5   회수       6788 non-null   int32         
 6   댓글수      6788 non-null   int32         
 7   추천수      6788 non-null   int32         
 8   평점       6788 non-null   float64       
dtypes: datetime64[ns](2), float64(1), int32(3), object(3)
memory usage: 397.9+ KB


In [14]:
# 라이트 노벨 제거
index=df.loc[(df.장르 == '라이트노벨') ].index
index
df.drop(index, inplace=True)

In [15]:
# 미스터리를 현대판타지로, bl를 로맨스로
df.loc[df.장르 == '미스터리', '장르'] = '현판'
df.loc[df.장르 == 'BL', '장르'] = '로맨스'

In [16]:
print(df.장르.value_counts())
df = df.reset_index(drop=True)
df.index = df.index+1
df

판타지    1735
로맨스    1412
현판     1331
무협     1306
로판      920
Name: 장르, dtype: int64


,제목,장르,작가,연재 시작일,마지막 연재일,회수,댓글수,추천수,평점
1,불로불사로 산다는 것은,판타지,겨울에어컨,2023-11-29,2024-06-27,196,1710,23,8.8
2,천마홈즈 런던앙복,무협,옴니버,2023-03-28,2024-06-27,394,5977,125,8.7
3,이름대로 살련다,현판,SSANTA,2023-12-06,2024-06-27,151,40,2,9.3
4,부패의 사제,판타지,사다듬,2020-12-17,2024-06-27,375,3587,116,8.8
5,불로불사가 사는 법,현판,Neuf,2023-01-24,2024-06-27,409,4169,30,9.3
...,...,...,...,...,...,...,...,...,...
6700,허공답보,무협,배금산,2013-01-21,2013-01-21,5,3,2,6.9
6701,매직 크리에이터,현판,이상규,2012-12-14,2012-12-14,6,7,9,7.6
6702,카르미스,판타지,케이츠,2011-10-27,2011-10-27,8,9,5,6.2
6703,만겁강호,무협,구당협,2015-03-19,2015-03-19,8,7,8,5.8


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6704 entries, 1 to 6704
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   제목       6704 non-null   object        
 1   장르       6704 non-null   object        
 2   작가       6704 non-null   object        
 3   연재 시작일   6704 non-null   datetime64[ns]
 4   마지막 연재일  6704 non-null   datetime64[ns]
 5   회수       6704 non-null   int32         
 6   댓글수      6704 non-null   int32         
 7   추천수      6704 non-null   int32         
 8   평점       6704 non-null   float64       
dtypes: datetime64[ns](2), float64(1), int32(3), object(3)
memory usage: 392.9+ KB


In [18]:
df.to_csv('./data/N_Normal.txt')